In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.clustering import KMeans,KMeansModel
from math import sqrt
from itertools import permutations
import numpy as np

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1605951452212_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark_conf = SparkConf()\
        .setAppName("batch_KMeans")
sc=SparkContext.getOrCreate(spark_conf) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
num_partitions = 16

file_size = "1GB"
s3_folder = "s3://vince-streaming-kmeans/"
s3_subfolder = "Kmeans_features_k3_f4_"+file_size+"/"

feature_paths = [s3_folder + s3_subfolder + "Kmeans_k3_f4_"+file_size+"_features"]*10
initCenters_path = s3_folder + s3_subfolder + "Kmeans_k3_f4_"+file_size+"_initCenters.csv"
trueCenters_path = s3_folder + s3_subfolder + "Kmeans_k3_f4_"+file_size+"_centers.csv"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
trainingData = sc.textFile(feature_paths[0]+"_1.csv", num_partitions)\
        .map(lambda line: Vectors.dense([float(x) for x in line.strip().split(',')])).persist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
n_t = trainingData.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
rawInitCenters = sc.textFile(initCenters_path)\
        .map(lambda line: [float(x) for x in line.strip().split(',')])

num_centers = 3
centerWeights = [1.0]*num_centers
initialCenters = rawInitCenters.collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
split_nums = 10

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
KM = KMeans.train(trainingData,3,initialModel=KMeansModel(initialCenters))
c_t = KM.clusterCenters

trainingData.unpersist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PythonRDD[2] at RDD at PythonRDD.scala:53

In [14]:
for i in range(1,split_nums):
    delta_data = sc.textFile(feature_paths[i]+"_"+str(i+1)+".csv", num_partitions)\
        .map(lambda line: Vectors.dense([float(x) for x in line.strip().split(',')])).persist()
    m_t = delta_data.count()
    KM_updated = KMeans.train(delta_data,3,initialModel=KMeansModel(c_t))
    x_t = KM_updated.clusterCenters
    
    new_center = []
    for p1, p2 in zip(c_t, x_t):
        new_point = []
        for param1, param2 in zip(p1, p2):
            new_param = (param1*n_t + param2*m_t)/(n_t+m_t)
            new_point.append(new_param)
        new_center.append(np.array(new_point))
    c_t = new_center
    n_t = n_t + m_t
    
    delta_data.unpersist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PythonRDD[18] at RDD at PythonRDD.scala:53
PythonRDD[31] at RDD at PythonRDD.scala:53
PythonRDD[44] at RDD at PythonRDD.scala:53
PythonRDD[57] at RDD at PythonRDD.scala:53
PythonRDD[70] at RDD at PythonRDD.scala:53
PythonRDD[83] at RDD at PythonRDD.scala:53
PythonRDD[96] at RDD at PythonRDD.scala:53
PythonRDD[109] at RDD at PythonRDD.scala:53
PythonRDD[122] at RDD at PythonRDD.scala:53

In [15]:
rawTrueCenters = sc.textFile(trueCenters_path)\
        .map(lambda line: [float(x) for x in line.strip().split(',')])
trueCenters = rawTrueCenters.collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
permutations_trueCenters = list(permutations(trueCenters,3))
SDs = [0] * len(permutations_trueCenters)
for m in range(len(permutations_trueCenters)):
    temp_trueCenters = permutations_trueCenters[m]
    for n in range(len(trueCenters)):
        SDs[m] += sqrt(sum([x**2 for x in (c_t[n] - temp_trueCenters[n])]))
SD = min(SDs)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
SD

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.0020102680156735206